In [31]:
import requests
import pandas as pd
from sqlalchemy import create_engine
import psycopg2

f = open('twitch_secrets.txt')
secret_data = f.read()
f.close()
secret_data

Client_ID = secret_data.split('\n')[0]
Client_Secret = secret_data.split('\n')[1]
Oauth_Token = secret_data.split('\n')[2]
PGE_Secret = secret_data.split('\n')[3]

sql = """
    SELECT tag_ids_arr FROM twitch_tag_key
    """

#INPUT YOUR OWN CONNECTION STRING HERE
conn_string = 'postgresql://postgres:'+ PGE_Secret +'@192.168.1.21:5432/postgres'

#perform to_sql test and print result
db = create_engine(conn_string)
conn = db.connect()

#execute query
pg_conn = psycopg2.connect(conn_string)
cur = pg_conn.cursor()
cur.execute(sql)
tags = cur.fetchall()

#Create a dataframe from the query results
tags_df = pd.DataFrame(tags, columns=['tag_ids_arr'])

#Convert the dataframe to a list and find all unique ids
tags_fin = tags_df['tag_ids_arr'].unique()


url = "https://api.twitch.tv/helix/tags/streams"

if len(tags_fin)//100 < 1:
    iters = 1
else:
    iters = len(tags_fin)//100

response = []

tags_chunks = [tags_fin[i*100:(i+1)*100] for i in range(iters)]
for i in range(iters):
    querystring = tags_chunks[i]

    #add "tag_id" to each element of querystring and convert to a string
    querystring = [('"tag_id": "'+str(x)+'"') for x in querystring]
    querystring = ','.join(querystring)

    #must be a string in the form of "tag_id": "number sequence from tag_id" for each tag_id needed up to 100 per request.


    headers = {
        'Client-ID' : Client_ID,
        'Authorization' : 'Bearer ' + Oauth_Token
        }

    response.append(requests.request("GET", url, headers=headers, params=querystring))



#Close the cursor and connection
pg_conn.commit()
cur.close()

#close connection
conn.close()

In [34]:
response[0].json()

{'data': [{'tag_id': '621fb5bf-5498-4d8f-b4ac-db4d40d401bf',
   'is_auto': False,
   'localization_names': {'bg-bg': 'Изчистване на 1 кредит',
    'cs-cz': '1 čistý kredit',
    'da-dk': '1 credit klaret',
    'de-de': 'Mit 1 Leben abschließen',
    'el-gr': '1 μόνο πίστωση',
    'en-us': '1 Credit Clear',
    'es-es': 'Sin continuaciones',
    'es-mx': 'Completar con 1 crédito',
    'fi-fi': 'Läpäisy ilman jatkamisia',
    'fr-fr': '1 seul continue',
    'hu-hu': '1 kredit rendben',
    'it-it': 'Completamento con un solo tentativo',
    'ja-jp': '1クレジットクリア',
    'ko-kr': '원코인 클리어',
    'nl-nl': 'Voltooien met 1 credit',
    'no-no': 'Fullført på ett forsøk',
    'pl-pl': 'Za 1 kredyt',
    'pt-br': 'Zerado com 1 crédito',
    'pt-pt': 'Terminar em 1 crédito',
    'ro-ro': '1 credit eliminat',
    'ru-ru': 'Прохождение с одной попытки',
    'sk-sk': '1 čistý kredit',
    'sv-se': 'Klara utan att använda Continue',
    'th-th': 'เล่นจบด้วย 1 เครดิต',
    'tr-tr': 'Tek Kredi Yeter',
   

In [35]:
tags_key = pd.DataFrame()

for i in range(iters):
    json_streams = response[i].json()
    json_df = pd.DataFrame(json_streams['data'])

    json_df['date_scraped'] = pd.to_datetime('now')
    tags_key = tags_key.append(json_df)
    #stream_df = json_df.drop(['thumbnail_url','title'], axis=1)

In [51]:
for row in tags_key.iterrows():
    row[1]['localization_names'] = row[1]['localization_names']['en-us']
tags_key.head()

{'bg-bg': 'Изчистване на 1 кредит', 'cs-cz': '1 čistý kredit', 'da-dk': '1 credit klaret', 'de-de': 'Mit 1 Leben abschließen', 'el-gr': '1 μόνο πίστωση', 'en-us': '1 Credit Clear', 'es-es': 'Sin continuaciones', 'es-mx': 'Completar con 1 crédito', 'fi-fi': 'Läpäisy ilman jatkamisia', 'fr-fr': '1 seul continue', 'hu-hu': '1 kredit rendben', 'it-it': 'Completamento con un solo tentativo', 'ja-jp': '1クレジットクリア', 'ko-kr': '원코인 클리어', 'nl-nl': 'Voltooien met 1 credit', 'no-no': 'Fullført på ett forsøk', 'pl-pl': 'Za 1 kredyt', 'pt-br': 'Zerado com 1 crédito', 'pt-pt': 'Terminar em 1 crédito', 'ro-ro': '1 credit eliminat', 'ru-ru': 'Прохождение с одной попытки', 'sk-sk': '1 čistý kredit', 'sv-se': 'Klara utan att använda Continue', 'th-th': 'เล่นจบด้วย 1 เครดิต', 'tr-tr': 'Tek Kredi Yeter', 'vi-vn': 'Chơi thắng chỉ cần 1 xèng', 'zh-cn': '一命通关', 'zh-tw': '一命通關'}
{'bg-bg': 'Жители на Буркина Фасо', 'cs-cz': 'Burkina Faso', 'da-dk': 'Burkina Faso', 'de-de': 'Burkinisch', 'el-gr': 'Κάτοικος Μπουρκ

,tag_id,is_auto,localization_names,localization_descriptions,date_scraped
0,621fb5bf-5498-4d8f-b4ac-db4d40d401bf,False,"{'bg-bg': 'Изчистване на 1 кредит', 'cs-cz': '...",{'bg-bg': 'За потоци с акцент върху завършване...,2021-08-29 10:26:35.197625
1,20986b7d-c78f-4a94-a798-563c136ae21e,False,"{'bg-bg': 'Жители на Буркина Фасо', 'cs-cz': '...","{'bg-bg': 'За поточни предавания и съдържание,...",2021-08-29 10:26:35.197625
2,e629f79f-a7ce-4934-a392-2495cbab7010,False,"{'bg-bg': 'Пикълбол', 'cs-cz': 'Pickleball', '...","{'bg-bg': 'За поточни предавания, които включв...",2021-08-29 10:26:35.197625
3,7b49f69a-5d95-4c94-b7e3-66e2c0c6f6c6,False,"{'bg-bg': 'Дизайн', 'cs-cz': 'Design', 'da-dk'...",{'bg-bg': 'За потоци с акцент върху креативния...,2021-08-29 10:26:35.197625
4,1c628b75-b1c3-4a2f-9d1d-056c1f555f0e,True,"{'bg-bg': 'Шампион: Lux', 'cs-cz': 'Šampion: L...",{'bg-bg': 'За потоци с шампиона Lux в League o...,2021-08-29 10:26:35.197625
